In [1]:
import openai
import os
import time
import pandas as pd
from pathlib import Path
from tqdm import tqdm

APIキーは以下のURLから取得できます。  
https://platform.openai.com/account/api-keys  

【注意】  
<font color="Crimson">APIキーが流出すると他人に利用されて料金を請求されます！</font>  
絶対に共有してはいけません。  

In [2]:
# APIキーの読み込み
# キーを格納したテキストファイルを指定して読み込み
with open(f'{Path.home()}/.private_info/api_key.txt') as f:
    openai.api_key = f.read()

# ChatGPT APIのリクエスト送信制限について  
  
時間当たりのトークン数とリクエスト数は上限が定められており、どちらか1つでも上限を超えるとRateLimitErrorになります。  
レート制限はユーザ属性で大きく異なります。  
イテレーションする際、トークン上限はあまり気にならないですがリクエスト上限は要注意です。  

【リクエスト上限】  
クレジットカード未登録：1分当たり3リクエストが上限  
クレジットカード登録から48時間：1分当たり35リクエストが上限  
クレジットカード登録から48時間後：1分当たり3500リクエストが上限  
[リミット詳細（公式）](https://platform.openai.com/docs/guides/rate-limits/overview)  

クレジットカード未登録ユーザでは実質イテレーション不可能なので、[こちら](https://platform.openai.com/account/billing/payment-methods)から登録推奨。  
クレジットカードを登録しても、配布された無料クレジットがある場合にはそちらから消費されます。  
クレジットカードを登録すると従量課金となり、トークン消費に応じて毎月料金を請求されます。  
高額請求が不安な場合は[こちら](https://platform.openai.com/account/billing/limits)から月の利用料上限を設定可能です。  

【トークン見積もり】  
1. [Tokenizer](https://platform.openai.com/tokenizer)  
テキストを入力するとトークン数を表示してくれる公式Webサービスです。  
2. [tiktoken](https://github.com/openai/tiktoken)  
OpenAIが提供しているライブラリを用いてトークン化すると、要素数がトークン数になります。  
```python
import tiktoken

tiktoken.encoding_for_model('gpt-3.5-turbo')
tokens = enc.encode('Hello World!')
print(len(tokens))
```
<br>

トークン量が上限を超える場合は、まず以下の対処を考えます。  
・チャットの往復を減らす（こまめにmessageをリセットする）  
・openai.ChatCompletion.createの引数max_tokenを指定して、モデルが生成するメッセージの最大トークン数を制限する  

In [3]:
# ChatGPT settings
def send_request(message: str, role: str, delay: float = None, **kwargs):
    """
    レート上限に引っかかる場合は、引数delay（秒）を指定し、リクエスト送信毎に待機時間を設ける。
    """
    if delay:
        time.sleep(delay)
    messages = []
    # system role
    if role:
        messages.append(
            {'role': 'system', 'content': role},
        )
    # input message
    messages.append(
        {'role': 'user', 'content': message},
    )
    # send request
    chat_completion = openai.ChatCompletion.create(
        model='gpt-3.5-turbo',
        messages=messages,
        **kwargs
    )
    # get reply
    reply = chat_completion.choices[0].message.content
    return reply

# CSV処理関数の作成

元のcsv

In [4]:
pd.read_csv('./data/questions.csv', encoding='utf-8-sig').head()

,role,message
0,日付に関する情報を読み取り回答せよ。回答形式は「1月1日9時0分」の場合は「1-1 9:00...,(７月２１日（水) １８時５４分)
1,日付に関する情報を読み取り回答せよ。回答形式は「1月1日9時0分」の場合は「1-1 9:00...,(８月１８日（水) １６時５９分)
2,日付に関する情報を読み取り回答せよ。回答形式は「1月1日9時0分」の場合は「1-1 9:00...,(８月２４日（火) １１時１８分)
3,日付に関する情報を読み取り回答せよ。回答形式は「1月1日9時0分」の場合は「1-1 9:00...,(８月２７日（金) １４時１１分)
4,日付に関する情報を読み取り回答せよ。回答形式は「1月1日9時0分」の場合は「1-1 9:00...,(９月３日（金) １４時)


In [5]:
def reply_to_csv(path: str,
                 message_col: str = 'message',
                 role_col: str = None,
                 delay: float = 3,
                 read_kwargs: dict = {},
                 gpt_kwargs: dict = {}) -> pd.core.frame.DataFrame:
    """
    message_col: ChatGPTに送信するメッセージ（user role）の列名
    role_col: system roleの列名（任意）
    System roleを設定しない場合はメッセージの内容のみから回答する
    """
    df = pd.read_csv(path, **read_kwargs)
    message_list = df.loc[:, message_col].values
    if role_col:
        role_list = df.loc[:, role_col].values
    else:
        role_list = [None] * len(message_list)
    assert len(message_list) == len(role_list), 'role列とmessage列の長さが異なります。揃えて再実行して下さい。'
    reply_list = []
    for role, message in zip(tqdm(role_list), message_list):
        reply = send_request(message=message, role=role, delay=delay, **gpt_kwargs)
        reply_list.append(reply)
    df['reply'] = reply_list
    # Save to csv
    root, ext = os.path.splitext(path)
    df.to_csv(f'{root}_reply{ext}', encoding='utf-8-sig', index=False)
    return df

In [6]:
df = reply_to_csv(path='./data/questions.csv',
                  message_col='message',
                  role_col='role',
                  delay=None,
                  read_kwargs=dict(encoding='utf-8-sig'),
                  gpt_kwargs=dict(temperature=0.7))

100%|██████████| 24/24 [00:51<00:00,  2.16s/it]


ChatGPTの回答付きcsv

In [7]:
df.head()

,role,message,reply
0,日付に関する情報を読み取り回答せよ。回答形式は「1月1日9時0分」の場合は「1-1 9:00...,(７月２１日（水) １８時５４分),7-21 18:54
1,日付に関する情報を読み取り回答せよ。回答形式は「1月1日9時0分」の場合は「1-1 9:00...,(８月１８日（水) １６時５９分),8-18 16:59
2,日付に関する情報を読み取り回答せよ。回答形式は「1月1日9時0分」の場合は「1-1 9:00...,(８月２４日（火) １１時１８分),8-24 11:18
3,日付に関する情報を読み取り回答せよ。回答形式は「1月1日9時0分」の場合は「1-1 9:00...,(８月２７日（金) １４時１１分),8-27 14:11
4,日付に関する情報を読み取り回答せよ。回答形式は「1月1日9時0分」の場合は「1-1 9:00...,(９月３日（金) １４時),9-3 14:00
